In [61]:
import duckdb

In [62]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL ducklake; LOAD ducklake;")

In [63]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';")
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

In [64]:
catalogMetadata = '../dataLakehouse/catalog.ducklake'

In [65]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS dataLakehouse;")
con.sql("USE dataLakehouse;")

BinderException: Binder Error: Failed to attach DuckLake MetaData "__ducklake_metadata_dataLakehouse" at path + "../dataLakehouse/catalog.ducklake"Unique file handle conflict: Cannot attach "__ducklake_metadata_dataLakehouse" - the database file "../dataLakehouse/catalog.ducklake" is already attached by database "__ducklake_metadata_dataLakehouse"

In [ ]:
con.sql("SELECT * FROM ducklake_snapshots('dataLakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬───────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                            changes                            │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                    map(varchar, varchar[])                    │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼───────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-11 16:54:27.968607+07 │              0 │ {schemas_created=[main]}                                      │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-11 16:54:28.080394+07 │              1 │ {schemas_created=[raw]}                                

In [ ]:
con.sql("SELECT * FROM information_schema.schemata;")

┌───────────────────────────────────┬──────────────────────────┬──────────────┬───────────────────────────────┬──────────────────────────────┬────────────────────────────┬──────────┐
│           catalog_name            │       schema_name        │ schema_owner │ default_character_set_catalog │ default_character_set_schema │ default_character_set_name │ sql_path │
│              varchar              │         varchar          │   varchar    │            varchar            │           varchar            │          varchar           │ varchar  │
├───────────────────────────────────┼──────────────────────────┼──────────────┼───────────────────────────────┼──────────────────────────────┼────────────────────────────┼──────────┤
│ __ducklake_metadata_dataLakehouse │ main                     │ duckdb       │ NULL                          │ NULL                         │ NULL                       │ NULL     │
│ __ducklake_metadata_dataLakehouse │ sqlmesh                  │ duckdb       │ NULL 

In [ ]:
con.sql("SHOW TABLES FROM dataLakehouse;")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ campaigns    │
│ customers    │
│ events       │
│ products     │
│ transactions │
└──────────────┘

In [ ]:
con.sql("SHOW TABLES FROM __ducklake_metadata_dataLakehouse;")

┌────────────────────────────────────────────────┐
│                      name                      │
│                    varchar                     │
├────────────────────────────────────────────────┤
│ _auto_restatements                             │
│ _environment_statements                        │
│ _environments                                  │
│ _intervals                                     │
│ _snapshots                                     │
│ _versions                                      │
│ ducklake_column                                │
│ ducklake_column_mapping                        │
│ ducklake_column_tag                            │
│ ducklake_data_file                             │
│       ·                                        │
│       ·                                        │
│       ·                                        │
│ ducklake_view                                  │
│ full_model                                     │
│ full_model                   

In [66]:
con.sql("SELECT * FROM __ducklake_metadata_dataLakehouse.sqlmesh__sqlmesh_example.sqlmesh_example__incremental_model__3572673861;")

BinderException: Binder Error: Catalog "__ducklake_metadata_dataLakehouse" does not exist!

In [ ]:
con.sql("SELECT * FROM raw.products;")

┌────────────┬─────────────┬───────────┬────────────┬─────────────┬────────────┐
│ product_id │  category   │   brand   │ base_price │ launch_date │ is_premium │
│   int64    │   varchar   │  varchar  │   double   │    date     │   int64    │
├────────────┼─────────────┼───────────┼────────────┼─────────────┼────────────┤
│          1 │ Grocery     │ Brand_58  │      14.19 │ 2021-08-02  │          0 │
│          2 │ Fashion     │ Brand_1   │       25.8 │ 2021-09-14  │          0 │
│          3 │ Electronics │ Brand_70  │     165.46 │ 2021-01-18  │          1 │
│          4 │ Fashion     │ Brand_56  │      75.45 │ 2023-03-03  │          1 │
│          5 │ Sports      │ Brand_1   │       72.5 │ 2022-04-19  │          1 │
│          6 │ Fashion     │ Brand_39  │      57.08 │ 2023-09-16  │          0 │
│          7 │ Beauty      │ Brand_42  │      27.23 │ 2023-05-17  │          0 │
│          8 │ Home        │ Brand_41  │     121.82 │ 2021-02-16  │          1 │
│          9 │ Home        │

In [ ]:
con.sql("SELECT * FROM raw.transactions;")

┌────────────────┬─────────────────────┬─────────────┬────────────┬──────────┬──────────────────┬───────────────┬─────────────┬─────────────┐
│ transaction_id │      timestamp      │ customer_id │ product_id │ quantity │ discount_applied │ gross_revenue │ campaign_id │ refund_flag │
│     int64      │      timestamp      │    int64    │   double   │  int64   │      double      │    double     │    int64    │    int64    │
├────────────────┼─────────────────────┼─────────────┼────────────┼──────────┼──────────────────┼───────────────┼─────────────┼─────────────┤
│              1 │ 2021-12-27 08:25:15 │       59540 │     1630.0 │        3 │              0.0 │         43.74 │           0 │           0 │
│              2 │ 2023-06-06 21:14:26 │       54871 │     1901.0 │        3 │              0.0 │        174.78 │          21 │           0 │
│              3 │ 2023-08-31 05:29:54 │       51818 │     1884.0 │        1 │              0.0 │         40.61 │          37 │           0 │
│     

In [67]:
con.close()